In [ ]:
'''
분산분석은 두개 이상의 다수 집단 간 평균을 피교하는 통계분석 방법으로
종속변수는 연속형, 독립변수는 범주형 변수이다. 
'''

### 1. 일원배치 분산분석(One-way ANOVA)
### type 1 : 하나의 연속형 특정컬럼과 하나의 범주형 특정컬럼을 분류하는 경우 

In [ ]:
import scipy.stats as stats
import pandas as pd
f_path = 
data = pd.read_csv(f_path)
data.head()

In [ ]:
col_outcome = # 범부형 종속변수 컬럼명 --> 범주형 값에 따른 독립변수 수치를 확인하는 용도
col_obj = # 연속형 독립변수 컬럼명
data[col_outcome].unique()

In [ ]:
data[col_outcome].value_counts()

In [ ]:
target_list = data[col_outcome].unique()
x0 = data[data[col_outcome]==target_list[0]][col_obj]
x1 = data[data[col_outcome]==target_list[1]][col_obj]
x2 = data[data[col_outcome]==target_list[2]][col_obj]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.scatterplot(x = col_outcome,
                y = col_obj,
                hue = col_outcome, #different colors by group
                style = col_outcome, #different shapes by group
                s = 100, # marker size
                data = data)
plt.show()

In [ ]:
print(stats.shapiro(x0))
print(stats.shapiro(x1))
print(stats.shapiro(x2))

In [ ]:
#1차 정규성검정
var_sha_x0 = stats.shapiro(x1)
var_sha_x1 = stats.shapiro(x1)
var_sha_x2 = stats.shapiro(x1)
if (var_sha_x0[1]<0.05)|(var_sha_x1[1]<0.05)|(var_sha_x2[1]<0.05):
    print('정규성을 따르지 않습니다.')
    print('kruskal 검정을 진행합니다.')
    anova = stats.kruskal(x0, x1, x2)
    if anova[1]>0.05:
        print('귀무가설(H0) 채택')
    else:
        print('대립가설(H1) 채택')
else:
    print('등분산 검정을 진행합니다.')

In [ ]:
#2차 등분산 검정
var_levene = stats.levene(x0, x1, x2)
if var_levene[1]<0.05:
    print('등분산성을 만족하지 못합니다.')
    #!pip3 install pingouin
    #conda install pingouin
    import pingouin as pg
    welch = pg.welch_anova(data=data, dv=col_obj, between=col_outcome)
    anova = welch.loc[0, "p-unc"]
    if anova > 0.05:
        print('귀무가설(H0) 채택')
    else:
        print('대립가설(H1) 채택')
else:
    print('정규성 및 등분산성 모두 만족합니다.')
    anova = stats.f_oneway(x1, x2, x3)
    if anova[1] > 0.05:
        print('귀무가설(H0) 채택')
    else:
        print('대립가설(H1) 채택')

In [ ]:
'''
사후검정이란, 분산분석의 결과 귀무가설이 기각되어 적어도 한 집단에서 평균의 차이가 있음이 통계적으로 증명,
어떤 집단들에 대해서 평균의 차이가 존재하는지 알아보기 위해 실시하는 분석이다. tuekeyhsd 방식을 주로 사용합니다.
예를 들면, tueykeyhsd.summary()를 통해서 p-adj 값이 0.05 이하일 경우 차이가 있음을 확인하는 경우다.
'''

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison
mc = MultiComparison(data=data[col_obj], groups = data[col_outcome])
tuekeyhsd = mc.tukeyhsd(alpha=0.05)
fig = tuekeyhsd.plot_simultaneous()

In [ ]:
tuekeyhsd.summary()

### 2. 일원배치 분산분석(Two-way ANOVA)

In [ ]:
'''
<가설방법>
1 - 교호작용 /  2,3 - 주효과 검정
귀무가설1(H0) : a와b변수의 상호작용 효과(교호작용, interaction)가 없다.
귀무가설2(H0) : a변수에 따른 종속변수의 값에는 차이가 없다.
귀무가설3(H0) : b변수에 따른 종속변수의 값에는 차이가 없다.
대립가설1(H1) : a와b변수의 상호작용 효과(교호작용, interaction)가 있다.
대립가설2(H1) : a변수에 따른 종속변수의 값에는 차이가 있다.
대립가설3(H1) : b변수에 따른 종속변수의 값에는 차이가 있다.

Python으로 양방향 ANOVA를 수행하는 방법
formula = y~C(x1)+C(x2)+C(x1):C(x2)
model = ols(formula, data).fit()
aov_table = anova_lm(model, typ=2)
'''

In [ ]:
# 사례1
import pandas as pd
import numpy as pd
f_path = 
data = pd.read_csv(f_path)
col_a = 
col_b = 
col_y = 
data = data[[cal_y, col_a, col_b]]
# 필요시 아래코드 적용
# data = data[(data[col_a]=='')|(data[col_a]=='')].reset_index(drop=True)

# 독립변수와 종속변수를 series로 변경
y_value = data[col_y]
a_value = data[col_a]
b_value = data[col_b]

#분산분석 수행
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
formula = f'{col_y} ~C({col_a})*C({col_b})'
model = ols(formula, data).fit()
tables =  anova_lm(model, typ=2)
'''
index별로 PR(>F)열 p-value값을 보고 귀무가설 기각/채택 여부를 판단, 0.05 유의수준 이하일 경우 상호작용효과 있고, 
차이가 있음을 의미한다.
'''

In [ ]:
#사례2

In [ ]:
import pandas as pd
f_path = 
data = pd.read_csv(f_path)
data.head()

In [ ]:
col_a = 
col_b =
col_y =
data = data[[col_y, col_a, col_b]]
data.info() # datatype, null info etc...

In [ ]:
# 분산분석 수행
#분산분석 수행
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
formula = f'{col_y} ~ C({col_a})+C({col_b})+C({col_a}):C({col_b})'
model = ols(formula, data).fit()
tables =  anova_lm(model, typ=2)